In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [2]:
sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

In [4]:
instacart = ss.read.csv('/Users/christopherolley/data/consolidated_df.csv', header=True, inferSchema=True)

In [6]:
instacart.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- eval_set: string (nullable = true)
 |-- order_number: integer (nullable = true)
 |-- order_dow: integer (nullable = true)
 |-- order_hour_of_day: integer (nullable = true)
 |-- days_since_prior_order: double (nullable = true)
 |-- product_id: double (nullable = true)
 |-- add_to_cart_order: double (nullable = true)
 |-- reordered: double (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- department: string (nullable = true)
 |-- aisle: string (nullable = true)



In [32]:
instacart.write.saveAsTable('Instacart')

# Product Features

Number of times a product was ordered:

In [14]:
instacart.filter("eval_set=='prior'").groupBy('product_id').count().orderBy("count", ascending=False).show()

+----------+------+
|product_id| count|
+----------+------+
|   24852.0|472565|
|   13176.0|379450|
|   21137.0|264683|
|   21903.0|241921|
|   47209.0|213584|
|   47766.0|176815|
|   47626.0|152657|
|   16797.0|142951|
|   26209.0|140627|
|   27845.0|137905|
|   27966.0|137057|
|   22935.0|113426|
|   24964.0|109778|
|   45007.0|104823|
|   39275.0|100060|
|   49683.0| 97315|
|   28204.0| 89632|
|    5876.0| 87746|
|    8277.0| 85020|
|   40706.0| 84255|
+----------+------+
only showing top 20 rows



Average position in cart:

In [19]:
instacart.filter("eval_set=='prior'").groupBy('product_id', 'product_name').avg("add_to_cart_order").orderBy("avg(add_to_cart_order)").show()

+----------+--------------------+----------------------+
|product_id|        product_name|avg(add_to_cart_order)|
+----------+--------------------+----------------------+
|   37088.0|Lndbrg White Quin...|                   1.0|
|   39759.0|Indoor & Outdoor ...|                   1.0|
|   44495.0|Lowfat Cherry Lim...|                   1.0|
|   25441.0|       Spicy Falafel|                   1.0|
|   41800.0|       Easter Basket|                   1.0|
|   43144.0|Flax Oil, Organic...|                   1.0|
|   33440.0|Rosa Mosqueta Ros...|                   1.0|
|    2769.0|Pappardelle Nests...|                   1.0|
|   12841.0|      King Crab Legs|                   1.0|
|   34962.0|  Prenatal Nutrients|                   1.0|
|   25888.0|Super Nutrition V...|                   1.0|
|   27863.0|        Cooking Fuel|                   1.0|
|   47680.0| Indian Wells Merlot|                   1.0|
|    2796.0|      American Blend|                   1.0|
|   36597.0|Vanilla Flavor Mu..

Reorder rate: number of reorders/total number of times ordered

# User x Product Features

Reorder count: number of time each user has reordered a product:

In [31]:
instacart.filter("eval_set=='prior'").groupBy('product_id', 'user_id').sum('reordered').show()

+----------+-------+--------------+
|product_id|user_id|sum(reordered)|
+----------+-------+--------------+
|   35547.0|    385|           0.0|
|   27845.0|    433|           1.0|
|   34050.0|    601|           4.0|
|   44949.0|    853|           9.0|
|   35221.0|   1123|           0.0|
|   13113.0|   1607|           2.0|
|   18224.0|   2187|           3.0|
|   30620.0|   2451|           0.0|
|    5612.0|   2736|           7.0|
|   33198.0|   2954|           0.0|
|   24852.0|   2991|           6.0|
|   15424.0|   3686|           2.0|
|   30779.0|   3781|           1.0|
|   45989.0|   4381|          10.0|
|   14705.0|   4630|           1.0|
|   35547.0|   5104|           4.0|
|   38387.0|   5117|           2.0|
|   37687.0|   5245|           2.0|
|    4987.0|   5387|           4.0|
|    6377.0|   5470|           1.0|
+----------+-------+--------------+
only showing top 20 rows



Number of orders since a user last ordered a given item: done by generating chrononological order_num from order_id, and returns max order_num (grouped by user) - max order_num (grouped by user and product)

In [99]:
ss.sql("select distinct product_id, user_id,\
        max(order_num) over (partition by user_id) - max(order_num) over (partition by user_id, product_id) as num_since_last_order_user_item from\
        (select Instacart.order_id, Instacart.user_id, Instacart.product_id, rhs.order_num\
        from Instacart\
        left join\
        (select order_id, user_id, row_number() over (partition by user_id order by order_id) as order_num from\
        (select distinct order_id, user_id from Instacart where eval_set = 'prior') as iq) as rhs\
        on Instacart.order_id=rhs.order_id and Instacart.user_id=rhs.user_id\
        where eval_set = 'prior') as iq2").show(50)

+----------+-------+------------------------------+
|product_id|user_id|num_since_last_order_user_item|
+----------+-------+------------------------------+
|     132.0|    148|                             1|
|     274.0|    148|                             4|
|     781.0|    148|                             3|
|    1203.0|    148|                             1|
|    1915.0|    148|                             1|
|    3376.0|    148|                             3|
|    4025.0|    148|                             5|
|    4477.0|    148|                             5|
|    5201.0|    148|                             1|
|    6891.0|    148|                             1|
|    7412.0|    148|                             1|
|    7539.0|    148|                             1|
|    7870.0|    148|                             1|
|    8362.0|    148|                             3|
|   10129.0|    148|                             0|
|   10451.0|    148|                             6|
|   10983.0|

Rate of user item reorder: # of reorders of an item / # of orders since first time ordering item.
Get max(order_num) grouped by user_id, then min(order_num) grouped by user_id and product, subtract the two to get number of orders since first purchase of an item. Then sum(reordered) grouped by item, user to get the number of times an item was reordered by a user

In [108]:
ss.sql("select product_id, user_id, num_reorders_product_user, since_first_order_user_product,\
        num_reorders_product_user/since_first_order_user_product as reorder_rate_product_user from\
        (select distinct product_id, user_id,\
        max(order_num) over (partition by user_id) - min(order_num) over (partition by user_id, product_id) as since_first_order_user_product,\
        sum(reordered) over (partition by user_id, product_id) as num_reorders_product_user from\
        (select Instacart.order_id, Instacart.user_id, Instacart.product_id, Instacart.reordered, rhs.order_num\
        from Instacart\
        left join\
        (select order_id, user_id, row_number() over (partition by user_id order by order_id) as order_num from\
        (select distinct order_id, user_id from Instacart where eval_set = 'prior') as iq) as rhs\
        on Instacart.order_id=rhs.order_id and Instacart.user_id=rhs.user_id\
        where eval_set = 'prior') as iq2) as iq3").show()

+----------+-------+-------------------------+------------------------------+-------------------------+
|product_id|user_id|num_reorders_product_user|since_first_order_user_product|reorder_rate_product_user|
+----------+-------+-------------------------+------------------------------+-------------------------+
|     132.0|    148|                      0.0|                             1|                      0.0|
|     274.0|    148|                      0.0|                             4|                      0.0|
|     781.0|    148|                      0.0|                             3|                      0.0|
|    1203.0|    148|                      0.0|                             1|                      0.0|
|    1915.0|    148|                      0.0|                             1|                      0.0|
|    3376.0|    148|                      0.0|                             3|                      0.0|
|    4025.0|    148|                      0.0|                  

# User Features

Maximum number of times a user has ordered any product:

In [54]:
ss.sql("select uid, max(ct) from\
       (select product_id as pid, user_id as uid, count(*) as ct from Instacart\
       where eval_set = 'prior'\
       group by 1,2) as iq\
       group by 1").show()

+------+-------+
|   uid|max(ct)|
+------+-------+
| 11141|      2|
| 46266|     10|
| 16386|     27|
|160492|     10|
|156366|     33|
| 11748|      4|
| 17679|     56|
|  7993|      7|
| 63271|     34|
|156197|      6|
| 57380|     14|
|  3794|     38|
|  4900|     13|
| 43103|      4|
| 43714|     16|
|  9376|     20|
|156749|      4|
| 43527|     30|
| 43302|      8|
|160563|      7|
+------+-------+
only showing top 20 rows



Average order size by user:

In [77]:
ss.sql("select user_id, avg(order_size) from\
        (select user_id, order_id, count(*) as order_size from Instacart\
        where eval_set = 'prior'\
        group by 1,2) as iq\
        group by 1").show()

+-------+------------------+
|user_id|   avg(order_size)|
+-------+------------------+
|    496| 5.512195121951219|
|   4101|              18.4|
|   5156|               9.3|
|   6654|              10.0|
|   9376| 12.13888888888889|
|  18800|               3.0|
|  19984|12.785714285714286|
|  20924|         10.046875|
|  44906|12.941176470588236|
|  46994| 7.029411764705882|
|  57370| 9.619047619047619|
|  60769|18.413793103448278|
|  63106|15.885714285714286|
| 155959| 6.333333333333333|
| 156366|13.886075949367088|
| 160563|14.909090909090908|
| 133189|14.909090909090908|
| 162321|11.181818181818182|
| 164962| 4.925925925925926|
| 167316| 6.777777777777778|
+-------+------------------+
only showing top 20 rows



Number of aisles purchased from:

In [80]:
ss.sql("select user_id, count(distinct aisle) from Instacart\
        where eval_set = 'prior'\
        group by 1").show()

+-------+---------------------+
|user_id|count(DISTINCT aisle)|
+-------+---------------------+
|   2659|                   59|
|   2866|                   19|
|   3794|                   64|
|   3918|                   40|
|   5518|                   27|
|   7554|                   41|
|  13289|                   45|
|  15447|                   27|
|  19553|                    7|
|  20924|                   66|
|  47283|                   58|
|  48254|                   52|
|  56640|                   35|
|  59355|                   31|
|  61051|                   16|
|  63155|                   36|
| 155042|                   81|
| 156527|                    9|
| 158257|                   45|
| 160767|                   21|
+-------+---------------------+
only showing top 20 rows



Number of departments purchased from:

In [82]:
ss.sql("select user_id, count(distinct department) from Instacart where eval_set = 'prior'\
        group by 1").show()

+-------+--------------------------+
|user_id|count(DISTINCT department)|
+-------+--------------------------+
|    148|                        15|
|   1591|                         9|
|   6620|                        11|
|  12799|                         7|
|  17420|                        15|
|  18024|                         7|
|  18051|                        10|
|  19079|                         8|
|  21220|                        15|
|  43935|                        15|
|  57039|                        11|
| 156296|                         9|
| 158593|                        12|
| 128389|                        14|
| 129153|                         7|
| 114851|                        19|
|  77803|                        13|
|  85749|                        10|
|  89476|                         9|
|  90019|                        14|
+-------+--------------------------+
only showing top 20 rows



Number of orders made by each user

In [103]:
ss.sql("select user_id, count(distinct order_id) from Instacart\
        where eval_set = 'prior'\
        group by 1").show()

+-------+------------------------+
|user_id|count(DISTINCT order_id)|
+-------+------------------------+
|    496|                      82|
|   4101|                       5|
|   5156|                      30|
|   6654|                       9|
|   9376|                      36|
|  18800|                       9|
|  19984|                      14|
|  20924|                      64|
|  44906|                      17|
|  46994|                      34|
|  57370|                      21|
|  60769|                      29|
|  63106|                      35|
| 155959|                      12|
| 156366|                      79|
| 160563|                      11|
| 133189|                      11|
| 162321|                      22|
| 164962|                      27|
| 167316|                      99|
+-------+------------------------+
only showing top 20 rows

